# Training step

Now that we have all the basic features, let's generated a simple model for topic modelling in order to discover what are these news about. For this experiment we will use the Topic Modeling with LDA, a simple clustering model that will help us investigating calssification features about data. This implementation is available at Gensim library. 

## Project setup

In [1]:
import sys
import os

sys.path.append('../')
os.chdir("../")

In [2]:
os.environ['DATA_DIR'] = os.path.join(os.path.abspath("./"), './data')

In [3]:
!wget -O - https://install.python-poetry.org | python3 -

--2022-12-01 15:10:36--  https://install.python-poetry.org/
Resolving install.python-poetry.org (install.python-poetry.org)... 76.76.21.98, 76.76.21.123
Connecting to install.python-poetry.org (install.python-poetry.org)|76.76.21.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28457 (28K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]  27.79K  --.-KB/s    in 0.001s  

2022-12-01 15:10:36 (22.4 MB/s) - written to stdout [28457/28457]

Retrieving Poetry metadata

The latest version (1.2.2) is already installed.


In [4]:
import os

os.environ['PATH'] = f"{os.environ['PATH']}:/home/jovyan/.local/bin"

In [5]:
!poetry env use system
!poetry env remove --all
!poetry config virtualenvs.create false

### Install dependencies

In [6]:
!poetry install --no-cache -n

Skipping virtualenv creation, as specified in config file.
Installing dependencies from lock file

No dependencies to install or update


### Project structure

As we are still following recommended [Data Science cookiecutter template](https://drivendata.github.io/cookiecutter-data-science/), the training step will be on `src/models` folder.

In [11]:
!mkdir -p src/models
!touch src/models/__init__.py

## Feature load step

Considering models composition strategy discussion on previous notebooks, we are going to use artifacts provided by `FeatureBuildFlow` in order to load results. In order to keep track of features and experiments, loading step will ned to know which version of the artifacts to load, so version tag will be used as input for this training step. 

**P.S.**: it is mandatory that training and feature building step use the same tag. So training will fail if there is no feature build step with the same tag.

In [36]:
%%writefile src/models/train_model.py
import os
import pandas as pd

from metaflow import FlowSpec, Parameter, step, Flow, current

class TrainModelFlow(FlowSpec):
    
    @step
    def start(self):
        version = list(current.tags)[0]
        print(f"Loading features from version {version}")
        fl = Flow('FeatureBuildFlow')
        r = list(fl.runs(version))[0]
        self.results = r.data.results
        self.next(self.end)
        
    @step
    def end(self):
        print("Done!")
        

if __name__ == '__main__':
    TrainModelFlow()

Overwriting src/models/train_model.py


In [37]:
!python src/models/train_model.py run --tag 0.0.1

Metaflow 2.7.14 executing TrainModelFlow for user:jovyan
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-12-01 16:00:18.164 Workflow starting (run-id 1669921218093386):
2022-12-01 16:00:18.190 [1669921218093386/start/1 (pid 3007)] Task is starting.
2022-12-01 16:00:18.608 [1669921218093386/start/1 (pid 3007)] Loading features from version 0.0.1
2022-12-01 16:00:18.799 [1669921218093386/start/1 (pid 3007)] Task finished successfully.
2022-12-01 16:00:18.834 [1669921218093386/end/2 (pid 3017)] Task is starting.
2022-12-01 16:00:19.246 [1669921218093386/end/2 (pid 3017)] Done!
2022-12-01 16:00:19.387 [1669921218093386/end/2 (pid 3017)] Task finished successfully.
2022-12-01 16:00:19.394 Done!


In [38]:
from metaflow import Flow
fl = Flow('TrainModelFlow')
r = fl.latest_run
df_r = r.data.results
df_r

,TITLE,CONTENT,LINK,PUBLISHED,PUBLISHED_DATE,token_set
0,"Com surto na região Norte, campanha contra o s...","<img src=""https://s2.glbimg.com/HA6WyXj_0FWazn...",https://g1.globo.com/ap/amapa/noticia/2018/07/...,2018-07-23T22:40:12+00:00,2018-07-23,"[vacinação, voltada, público, infantil, aconte..."
1,Polícia Civil de Juiz de Fora recebe denúncia ...,Duas firmas são de São Paulo e uma de Belo Hor...,https://g1.globo.com/mg/zona-da-mata/noticia/2...,2018-07-23T21:01:26+00:00,2018-07-23,"[duas, firmas, paulo, belo, horizonte, agora, ..."
2,Segunda edição do ‘Encontro de Bateristas do T...,"<img src=""https://s2.glbimg.com/ygBGnNNFsGwUPA...",https://g1.globo.com/mg/triangulo-mineiro/noti...,2018-07-23T20:45:48+00:00,2018-07-23,"[170, bateristas, esperados, evento, pode, tor..."
3,Comissariado do AP fiscaliza embarque de menor...,"<img src=""https://s2.glbimg.com/3ywfLG3crqb2_I...",https://g1.globo.com/ap/amapa/noticia/2018/07/...,2018-07-23T20:36:37+00:00,2018-07-23,"[fluxo, passageiros, aumenta, principais, port..."
4,Ceará tem 66 municípios com emergência reconhe...,"<img src=""https://s2.glbimg.com/aktFEMxx84AuWQ...",https://g1.globo.com/ce/ceara/noticia/2018/07/...,2018-07-23T20:13:49+00:00,2018-07-23,"[nesta, segunda-feira, reconhecimento, municíp..."
...,...,...,...,...,...,...
175,Por que a esquadria de alumínio se tornou um d...,"<img src=""https://s2.glbimg.com/k5A5kFIGvVX5Ib...",https://g1.globo.com/ba/bahia/especial-publici...,2022-11-28T18:40:41+00:00,2022-11-28,"[aliando, versatilidade, design, resistência, ..."
176,Perfil da PM em SC curte posts antidemocrático...,"<img src=""https://s2.glbimg.com/hHaPhjo7AiNLn-...",https://g1.globo.com/sc/santa-catarina/noticia...,2022-11-28T18:39:29+00:00,2022-11-28,"[polícia, militar, rodoviária, instaurou, proc..."
177,Ponte que liga Toca da Onça a Rio Bonito é rec...,"<img src=""https://s2.glbimg.com/iA1ARsUvBzuFOH...",https://g1.globo.com/rj/regiao-serrana/noticia...,2022-11-28T18:39:25+00:00,2022-11-28,"[ponte, existia, local, levada, força, água, r..."
178,Menina denuncia estupro à professora e ex-padr...,"<img src=""https://s2.glbimg.com/DsJJ0ftZ_UF0g9...",https://g1.globo.com/go/goias/noticia/2022/11/...,2022-11-28T18:38:07+00:00,2022-11-28,"[garota, anos, disse, abusada, desde, anos, se..."


## Training step

In [16]:
# FIXME: there is an install deprecation notice on gensim that prevents us using poetry to install it: https://github.com/RaRe-Technologies/gensim/issues/3362

!poetry run python -m pip install gensim --disable-pip-version-check --no-deps --no-cache-dir --no-binary gensim

Skipping virtualenv creation, as specified in config file.
DEPRECATION: --no-binary currently disables reading from the cache of locally built wheels. In the future --no-binary will not influence the wheel cache. pip 23.1 will enforce this behaviour change. A possible replacement is to use the --no-cache-dir option. You can use the flag --use-feature=no-binary-enable-wheel-cache to test the upcoming behaviour. Discussion can be found at https://github.com/pypa/pip/issues/11453


In [17]:
!poetry add gensim

Skipping virtualenv creation, as specified in config file.
The following packages are already present in the pyproject.toml and will be skipped:

  • gensim

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [52]:
%%writefile src/models/train_model.py
import os
import pandas as pd

import gensim

from metaflow import FlowSpec, Parameter, step, Flow, current

class TrainModelFlow(FlowSpec):
    topics = Parameter('topics', default=None)
    iterations = Parameter('iterations', default=100)

    @property
    def version(self):
        return list(current.tags)[0]
    
    @step
    def start(self):
        print(f"Loading features from version {self.version}")
        fl = Flow('FeatureBuildFlow')
        r = list(fl.runs(self.version))[0]
        self.results = r.data.results
        self.next(self.train_lda)

    @step 
    def train_lda(self):
        print(f"Training LDA model version {self.version}...")
        dictionary = gensim.corpora.Dictionary(self.results['token_set'])
        self.corpus_train = self.results['token_set'].apply(dictionary.doc2bow)
        self.lda_model = gensim.models.LdaMulticore(self.corpus_train, id2word=dictionary, iterations=self.iterations, num_topics=self.topics, random_state=42)
        self.next(self.end)
    
    @step
    def end(self):
        print("Done!")
        

if __name__ == '__main__':
    TrainModelFlow()

Overwriting src/models/train_model.py


### Registering parameters

Now we can use metaflow `Parameter` class to register model parameters. This will be important as we compare experiments later.

In [53]:
!python src/models/train_model.py run --tag 0.0.1 --topics 10 --iterations 100

Metaflow 2.7.14 executing TrainModelFlow for user:jovyan
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-12-01 16:12:47.002 Workflow starting (run-id 1669921966934743):
2022-12-01 16:12:47.029 [1669921966934743/start/1 (pid 3551)] Task is starting.
2022-12-01 16:12:47.625 [1669921966934743/start/1 (pid 3551)] Loading features from version 0.0.1
2022-12-01 16:12:47.875 [1669921966934743/start/1 (pid 3551)] Task finished successfully.
2022-12-01 16:12:47.908 [1669921966934743/train_lda/2 (pid 3561)] Task is starting.
2022-12-01 16:12:48.463 [1669921966934743/train_lda/2 (pid 3561)] Training LDA model version 0.0.1...
2022-12-01 16:12:48.958 [1669921966934743/train_lda/2 (pid 3561)] Task finished successfully.
2022-12-01 16:12:48.999 [1669921966934743/end/3 (pid 3580)] Task is starting.
2022-12-01 16:12:49.564 [1669921966934743/end/3 (pid 3580)] Done!
2022-12-01 16:12:49.732 [1669921966934743/end/3 (pid 3580)] Task finished successfully.
2022-

### Log model performance

Topic modeling performance can be calculated using Coherence Model. This will be registered in order to compare to next versions.

In [54]:
%%writefile src/models/train_model.py
import os
import pandas as pd

import gensim

from metaflow import FlowSpec, Parameter, step, Flow, current

class TrainModelFlow(FlowSpec):
    topics = Parameter('topics', default=None)
    iterations = Parameter('iterations', default=100)
    coherence_alg = Parameter('coherence', default='u_mass')

    @property
    def version(self):
        return list(current.tags)[0]
    
    @step
    def start(self):
        print(f"Loading features from version {self.version}")
        fl = Flow('FeatureBuildFlow')
        r = list(fl.runs(self.version))[0]
        self.results = r.data.results
        self.next(self.train_lda)

    @step 
    def train_lda(self):
        print(f"Training LDA model version {self.version}...")
        dictionary = gensim.corpora.Dictionary(self.results['token_set'])
        self.corpus_train = self.results['token_set'].apply(dictionary.doc2bow)
        self.lda_model = gensim.models.LdaMulticore(self.corpus_train, id2word=dictionary, iterations=self.iterations, num_topics=self.topics, random_state=42)
        self.next(self.end)
    
    @step
    def end(self):
        cm = gensim.models.coherencemodel.CoherenceModel(model=self.lda_model, corpus=self.corpus_train, coherence=self.coherence_alg)
        self.coherence = cm.get_coherence()
        print(f"Done! Model coherence is {self.coherence}")
        

if __name__ == '__main__':
    TrainModelFlow()

Overwriting src/models/train_model.py


In [55]:
!python src/models/train_model.py run --tag 0.0.1 --topics 10 --iterations 100

Metaflow 2.7.14 executing TrainModelFlow for user:jovyan
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-12-01 16:19:39.424 Workflow starting (run-id 1669922379345632):
2022-12-01 16:19:39.455 [1669922379345632/start/1 (pid 3796)] Task is starting.
2022-12-01 16:19:40.010 [1669922379345632/start/1 (pid 3796)] Loading features from version 0.0.1
2022-12-01 16:19:40.246 [1669922379345632/start/1 (pid 3796)] Task finished successfully.
2022-12-01 16:19:40.284 [1669922379345632/train_lda/2 (pid 3806)] Task is starting.
2022-12-01 16:19:40.836 [1669922379345632/train_lda/2 (pid 3806)] Training LDA model version 0.0.1...
2022-12-01 16:19:41.339 [1669922379345632/train_lda/2 (pid 3806)] Task finished successfully.
2022-12-01 16:19:41.382 [1669922379345632/end/3 (pid 3825)] Task is starting.
2022-12-01 16:19:42.009 [1669922379345632/end/3 (pid 3825)] Done! Model coherence is -6.194980314046008
2022-12-01 16:19:42.216 [1669922379345632/end/3 (pid 38